In [1]:
# IMPORTING LIBRARIES REQUIRED FOR THIS CODE
import sys
import numpy as np
from scipy.stats import randint
import scipy.io as sio
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn.model_selection import train_test_split
# from sklearn.cross_validation import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import pipeline
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [2]:
#from keras.processing import sequence
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
#from keras.layers import embedding
from tensorflow.keras.models  import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD 
from tensorflow.keras.callbacks import EarlyStopping
#from keras.utils import np_utils
import itertools
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D

In [3]:
# loading input and output file
# the loaded filecontaints 500 x 5 points at input
# to list - frequency points and the corresponding autospectra
dfin = sio.loadmat('5paracnn5crawdisjointlogspacefreq.mat')['infile']
dfout = sio.loadmat('5paracnn5crawdisjointlogspacefreq.mat')['outfile']

In [4]:

Shuffling the datasets to vary the splits used for cross-validation
ind = np.random.RandomState(22).permutation(range(len(dfin)))

# split into input and outputs
dfin = dfin[ind][:][:]
dfout = dfout[ind][:][:]
 

In [5]:
# loss function with error for parameter alpha
def pwwloss(y_act, y_pred):
    a = [0.3734, 0.1996, 0.1224]
    a1 = K.abs(0.3734*y_act[0] + 0.1996*y_act[1] + 0.1224*y_act[2] - 0.3734*y_pred[0] - 0.1996*y_pred[1] - 0.1224*y_pred[2])
    print(a1)
    b1 = K.sum(K.abs(y_act - y_pred))
    print(b1)
    return a1 + b1

In [6]:
# Loss  function with higher weightage for error in time constant
def pTTl(y_act, y_pred):
    a = [0.3734, 0.1996, 0.1224]
    a1 = K.abs(0.9*y_act[0] + 0.9*y_act[1] + 0.9*y_act[2] - 0.9*y_pred[0] - 0.9*y_pred[1] - 0.9*y_pred[2])
    print(a1)
    b1 = K.sum(K.abs(y_act - y_pred))
    print(b1)
    return -a1 + b1

In [7]:
nk = 0
n = str(nk)

In [8]:
# Appropriate loss dunction can be slected for training. 

for train_index, test_index in KFold(n_splits=5).split(dfin):
    train_X, test_X = dfin[train_index][:][:], dfin[test_index][:][:]
    train_y, test_y = dfout[train_index][:][:], dfout[test_index][:][:]
    print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
    
    best_stop = [
    tf.keras.callbacks.EarlyStopping(patience=100, monitor = 'val_loss', verbose = 1),
    tf.keras.callbacks.ModelCheckpoint(filepath='models/best_apt_model' + n + '.h5', save_best_only = True),
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
    ]
    
    model = Sequential()
    model.add(Conv1D(filters =64, kernel_size =9, activation = 'sigmoid', input_shape=(500,5)))
    model.add(MaxPooling1D(pool_size=10,strides=10))
    model.add(Dropout(0.5))
    
    
    model.add(Flatten())
    model.add(Dense(4))
    epochs = 3000
    batch_size = 10
    model.compile(loss = 'mean_squared_error', optimizer = 'adam')
    print(model.summary())
    
    
    pww = model.fit(train_X, train_y, epochs=epochs, batch_size=batch_size, validation_data = (test_X,test_y), shuffle = 'False', callbacks = best_stop)
    model.load_weights('models/best_apt_model'+n+'.h5')
    yhat = model .predict(test_X)
    testset = ind[test_index]
    sio.savemat('Saved_outfiles/22-kf-20-2kTlosscnn'+n+'.mat',{'yhat':yhat, 'test_y' :test_y, 'testset' :testset})
    nk = nk+1
    n = str(nk)

(294, 500, 5) (294, 4) (74, 500, 5) (74, 4)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 492, 64)           2944      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 49, 64)            0         
_________________________________________________________________
dropout (Dropout)            (None, 49, 64)            0         
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dense (Dense)                (None, 4)                 12548     
Total params: 15,492
Trainable params: 15,492
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3000
30/30 [==============================] - 0s 11ms/step - 

KeyboardInterrupt: 